# December 04

Today's puzzle is very similar to the problem John and Rikk has recently worked on. 

We need to build a timeline of states, where a Elf is either awake or asleep. 

In [ ]:
import regex as re
import numpy as np
import pandas as pd

In [ ]:
# Sample data as provided in question
data= """
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
""".strip().splitlines()

In [ ]:
# We're now going to start storing the data in a file to avoid having to paste into the main document
# Make sure you create a file with the name below and save the real problem output there. 
# If you want to run just the sample data, skip this block
with open("./04-kws.txt", "r") as FILE:
    data = FILE.read().strip().splitlines()

In [ ]:
data[:5]

In the real data the 'events' are not sorted, so before we do anything else, we need to sort the list. The wonderful thing about ISO8601 order date format is that the dates sort alphanumerically. Keep that in mind whenever putting a date in filenames!

So to put this in chronological order, we simply need to sort the list.

In [ ]:
data.sort()
data[:5]

In [ ]:
guard_on_duty = None

sleep_times = dict()

pattern_guard_change = re.compile(".*Guard #(\d+) begins shift")
pattern_date = re.compile("\[(\d\d\d\d-\d\d-\d\d) (\d\d):(\d\d)\].*")
for line in data:
    # Check if the current line is a guard change
    match = pattern_guard_change.match(line)
    if match:
        guard_on_duty = match[1]
        continue
        
    # If not a guard change, then it has to be an awake or sleep line
    match = pattern_date.match(line)
    date = match[1]
    minutes = int(match[3])
    
    date_key = "{}|{}".format(date,guard_on_duty)
    sleep_array = sleep_times.get(date_key, np.zeros(60))
        
    # If instead it's a guard that falls asleep
    if "asleep" in line:
        sleep_array[minutes:] = 1
    elif "wakes up" in line:
        sleep_array[minutes:] = 0
        
    sleep_times[date_key] = sleep_array

df_sleep = pd.DataFrame.from_dict(sleep_times,orient="index")

# Split date and elf into separate index columns
df_sleep["date"] = df_sleep.index.map(lambda x: x.split("|")[0])
df_sleep["elf"] = df_sleep.index.map(lambda x: x.split("|")[1])
df_sleep = df_sleep.set_index(["date","elf"])

df_sleep

First we want to find the elf that falls asleep the most. We first figure out how much each elf slept in total for each minute of the hour.

In [ ]:
df_total = df_sleep.groupby('elf').sum()
df_total.head()

Next we want to know how much each elf slept in total

In [ ]:
df_total["total_min_asleep"] =  df_total[list(range(0,60))].sum(axis=1)
df_total.head()

We want to know in which minute each elf is most likely to be asleep. So we're looking for the column with the highest value for each elf.

We can use the [idxmax()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.idxmax.html) function to find the ID of the column with the highest value

In [ ]:
df_total["most_common_sleep_minute"] = df_total[list(range(0,60))].idxmax(axis=1)
df_total.head()

We can use the [max()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.max.html) function to find the corresponding maximum column value.

In [ ]:
df_total["most_common_sleep_minute_value"] = df_total[list(range(0,60))].max(axis=1)
df_total.head()

Finally, the question asks for the product of the minutes asleep in this minute with the ID of the elf, so we also create a `solution` column to hold this value.

In [ ]:
df_total["solution"] = df_total.index.astype(int) * df_total["most_common_sleep_minute"]
df_total.head()

> Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

So we just need to look at the highest value of `total_min_asleep`. We can sort and use the [head()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to find the top row.

In [ ]:
df_total[["total_min_asleep","most_common_sleep_minute","most_common_sleep_minute_value","solution"]]\
  .sort_values(by="total_min_asleep", ascending=False).head(1)

# Part 2

> Of all guards, which guard is most frequently asleep on the same minute?

We have already answered this above, so we just need to sort by `most_common_sleep_minute_value`

In [ ]:
df_total[["total_min_asleep","most_common_sleep_minute","most_common_sleep_minute_value", "solution"]]\
  .sort_values(by="most_common_sleep_minute_value", ascending=False).head(1)